In [2]:
import datetime, os, requests, time
from dateutil import tz

# TODO: As of July 2019, this hasn't been working for a while
# Complaints of no route to host.  Did the host change?
# Have we been blacklisted?


http://aqsdr1.epa.gov/aqsweb/aqstmp/airdata/download_files.html

https://ofmext.epa.gov/AQDMRS/ws/list?name=param&pc=CORE_HAPS&resource=rawData :

`12103	Arsenic (TSP) STP
12105	Beryllium (TSP) STP
12110	Cadmium (TSP) STP
12112	Chromium (TSP) STP
12115	Chromium VI (TSP) STP
12128	Lead (TSP) STP
12132	Manganese (TSP) STP
12136	Nickel (TSP) STP
14115	Chromium VI (TSP) LC
43218	1,3-Butadiene
43502	Formaldehyde
43503	Acetaldehyde
43505	Acrolein - Unverified
43509	Acrolein - Verified
43802	Dichloromethane
43803	Chloroform
43804	Carbon tetrachloride
43815	Ethylene dichloride
43817	Tetrachloroethylene
43818	1,1,2,2-Tetrachloroethane
43824	Trichloroethylene
43829	1,2-Dichloropropane
43830	trans-1,3-Dichloropropene
43831	cis-1,3-Dichloropropene
43843	Ethylene dibromide
43860	Vinyl chloride
45201	Benzene
82103	Arsenic PM10 STP
82105	Beryllium PM10 STP
82110	Cadmium PM10 STP
82112	Chromium PM10 STP
82128	Lead PM10 STP
82132	Manganese PM10 STP
82136	Nickel PM10 STP
82142	Mercury PM10 STP
88103	Arsenic PM2.5 LC
88105	Beryllium PM2.5 LC
88110	Cadmium PM2.5 LC
88112	Chromium PM2.5 LC
88128	Lead PM2.5 LC
88132	Manganese PM2.5 LC
88136	Nickel PM2.5 LC`

In [3]:
categories = {'44201': 'Ozone',
              '42401': 'SO2',
              '42101': 'CO',
              '42602': 'NO2',
              '88101': 'PM2.5', # federal reference method
              '88502': 'PM2.5', # non-FRM
              '81102': 'PM10',
              'SPEC':  None,
              'WIND':  None,
              'TEMP':  None,
              'PRESS': None,
              'RH_DP': None,
              'HAPS':  None,
              'VOCS':  None,
              'LEAD':  'Lead'}

years = range(1990, datetime.datetime.utcnow().year + 1)

src_dir = 'http://aqsdr1.epa.gov/aqsweb/aqstmp/airdata/'
dest_dir = 'EPA-AirData/'

# Last-Modified: Thu, 19 Jun 2014 17:28:44 GMT
# If-Modified-Since: Thu, 19 Jun 2014 17:28:44 GMT
def mirror_file_using_modtime(src_url, dest):
    headers = {}
    # If destination already exists, mirror only if newer
    try:
        stat = os.stat(dest)
        date = time.strftime('%a, %d %b %Y %H:%M:%S GMT', time.gmtime(stat.st_mtime))
        headers['If-Modified-Since'] = date
    except:
        pass
    
    
    response = requests.get(src_url, headers=headers)
    if response.status_code == 200:
        data = response.content
    
        server_modtime = dateutil.parser.parse(response.headers['Last-Modified'])
        server_modtime_epoch = datetime2epoch(server_modtime)
        tmp = dest + '.tmp' + str(os.getpid())
        os.makedirs(os.path.dirname(tmp), exist_ok=True)
        open(tmp, 'wb').write(data)
        os.rename(tmp, dest)
        os.utime(dest, (server_modtime_epoch, server_modtime_epoch))
        print('Wrote %d bytes to %s' % (len(data), dest))
    else:
        print('Received status code %d while fetching %s.  Skipping' % (response.status_code, src_url))

        

In [4]:
try:
    os.mkdir(dest_dir)
except:
    pass

for year in years:
    for category in categories:
        filename = 'hourly_%s_%d.zip' % (category, year)
        mirror_file_using_modtime(src_dir + filename, dest_dir + filename)

ConnectionError: HTTPConnectionPool(host='aqsdr1.epa.gov', port=80): Max retries exceeded with url: /aqsweb/aqstmp/airdata/hourly_44201_1990.zip (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7faa3a15eef0>: Failed to establish a new connection: [Errno 113] No route to host'))